## This notebook contains my scratch-work for the development of what is to follow. 

To begin, I played around with Azure AI's Document Intelligence capabilities. Here's some code trying out Azure AI's generic document processing. 

In [2]:
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

endpoint = "https://beachpoint-case.cognitiveservices.azure.com/"
key = "8bVXMQDHEioJ3neUYwBPrRsyF6n571OEMExO8QEcV7O0khHasuBNJQQJ99AKAC4f1cMXJ3w3AAALACOGDDTR"

client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(key))

with open("data/apple_financial_statement.pdf", "rb") as document:
    poller = client.begin_analyze_document("prebuilt-document", document)
    result = poller.result()

print(result)
for document in result.documents:
    print(f"Document type: {document.doc_type}")
    for name, field in document.fields.items():
        print(f"Field: {name}, Value: {field.value}, Confidence: {field.confidence}")


AnalyzeResult(api_version=2023-07-31, model_id=prebuilt-document, content=Apple Inc.
CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS (Unaudited) (In millions, except number of shares, which are reflected in thousands, and per-share amounts)
Three Months Ended
Twelve Months Ended
September 28, 2024
September 30, 2023
September 28, September 30,
2024
2023
Net sales:
Products
$ 69,958
$ 67,184
$ 294,866
$ 298,085
Services
24,972
22,314
96,169
85,200
Total net sales (1)
94,930
89,498
391,035
383,285
Cost of sales:
Products
44,566
42,586
185,233
189,282
Services
6,485
6,485
25,119
24,855
Total cost of sales
51,051
49,071
210,352
214,137
Gross margin
43,879
40,427
180,683
169,148
Operating expenses:
Research and development
7,765
7,307
31,370
29,915
Selling, general and administrative
6,523
6,151
26,097
24,932
Total operating expenses
14,288
13,458
57,467
54,847
Operating income
29,591
26,969
123,216
114,301
Other income/(expense), net
19
29
269
(565)
Income before provision for income taxes

Later down the way, I realized the importance of prompting the LLM to obtain structured data (this effectively reduces hallucination and forces the model to extract only the desired information). Here's code experimenting with OpenAI's APIs for structured LLM output.

In [ ]:
import openai
from openai import OpenAI
import os
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
import openpyxl
from prompt import FinancialStatementExtract
import json

from dotenv import load_dotenv

load_dotenv()

AZURE_DOCUMENT_ANALYZER_KEY = os.getenv("AZURE_DOCUMENT_ANALYZER_KEY")
AZURE_DOCUMENT_ANALYZER_ENDPOINT = os.getenv("AZURE_DOCUMENT_ANALYZER_ENDPOINT")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

document_client = DocumentAnalysisClient(
    endpoint=AZURE_DOCUMENT_ANALYZER_ENDPOINT,
    credential=AzureKeyCredential(AZURE_DOCUMENT_ANALYZER_KEY)
)
openai_client = OpenAI(api_key=OPENAI_API_KEY)

'''
This function takes in a PDF file path, extracts tables from the PDF
using Azure Document Analysis and returns the tables as a list of rows.
'''
def extract_tables_from_pdf(pdf_path):
    try:
        with open(pdf_path, "rb") as pdf_file:
            poller = document_client.begin_analyze_document("prebuilt-layout", document=pdf_file)
            result = poller.result()

        tables = []
        for table in result.tables:
            rows = []
            for row_idx in range(table.row_count):
                row = []
                for cell in table.cells:
                    if cell.row_index == row_idx:
                        row.append(cell.content)
                rows.append(row)
            tables.append(rows)
        return tables
    except Exception as e:
        print(f"Error extracting tables: {e}")
        return []

'''
This function takes in the parsed tables and prompts OpenAI
to extract the desired information and format it. 
'''
def parse_tables_with_openai(tables):
    with open('data/prompt.txt', 'r') as f:
        prompt = f.read()
    
    for table in tables:
        prompt += "\n".join(["\t".join(row) for row in table]) + "\n\n"

    try:
        response = openai_client.beta.chat.completions.parse(
            model="gpt-4o-2024-08-06", 
            messages=[
                {"role": "system", "content": "You are a helpful assistant for structured financial data extraction. You will be given unstructured text from a research paper and should convert it into the given structure."},
                {"role": "user", "content": prompt}
            ],
            response_format=FinancialStatementExtract
        )
        return response.choices[0].message.parsed
    except Exception as e:
        print(f"Error parsing tables with OpenAI: {e}")
        return ""


pdf_path = "data/apple_financial_statement.pdf"  
output_path = "data/apple_financial_metrics.xlsx"

print("Extracting tables from PDF...")
tables = extract_tables_from_pdf(pdf_path)
print(f"Hello world! {tables}")
print("Parsing tables with OpenAI...")

parsed_data = parse_tables_with_openai(tables)

glob_items = parsed_data
print(type(glob_items))

for key, value in glob_items:
    print(f"type of key: {type(key)}, type of value: {type(value)}")
    print(f"KEY: {key}, VALUE: {value}")

    vals = value.split('|')
    vals.append('N/A')
    
    print(vals)
    if value == 'N/A':
        print([str(key), 'N/A', 'N/A'])
    else:
        print([str(key), vals[0], vals[1]])
    print()

Extracting tables from PDF...
Hello world! [[['', 'Three Months Ended', 'Twelve Months Ended'], ['September 28, 2024', 'September 30, 2023', 'September 28,\n2024', 'September 30,\n2023'], ['Net sales:', '', '', '', ''], ['Products', '$ 69,958', '$ 67,184', '$ 294,866', '$ 298,085'], ['Services', '24,972', '22,314', '96,169', '85,200'], ['Total net sales (1)', '94,930', '89,498', '391,035', '383,285'], ['Cost of sales:', '', '', '', ''], ['Products', '44,566', '42,586', '185,233', '189,282'], ['Services', '6,485', '6,485', '25,119', '24,855'], ['Total cost of sales', '51,051', '49,071', '210,352', '214,137'], ['Gross margin', '43,879', '40,427', '180,683', '169,148'], ['', '', '', '', ''], ['Operating expenses:', '', '', '', ''], ['Research and development', '7,765', '7,307', '31,370', '29,915'], ['Selling, general and administrative', '6,523', '6,151', '26,097', '24,932'], ['Total operating expenses', '14,288', '13,458', '57,467', '54,847'], ['', '', '', '', ''], ['Operating income', '

In [9]:
for key, value in glob_items:
    print(f"type of key: {type(key)}, type of value: {type(value)}")
    print(f"KEY: {key}, VALUE: {value}")

    vals = value.split('|')
    vals.append('N/A')
    
    print(vals)
    if value == 'N/A':
        print([str(key), 'N/A', 'N/A'])
    else:
        print([str(key), vals[0], vals[1]])
    print()

type of key: <class 'str'>, type of value: <class 'str'>
KEY: Income_statement_millions, VALUE: $391,035M| $383,285M
['$391,035M', ' $383,285M', 'N/A']
['Income_statement_millions', '$391,035M', ' $383,285M']

type of key: <class 'str'>, type of value: <class 'str'>
KEY: Revenue_Item_1, VALUE: N/A
['N/A', 'N/A']
['Revenue_Item_1', 'N/A', 'N/A']

type of key: <class 'str'>, type of value: <class 'str'>
KEY: Revenue_Item_2, VALUE: N/A
['N/A', 'N/A']
['Revenue_Item_2', 'N/A', 'N/A']

type of key: <class 'str'>, type of value: <class 'str'>
KEY: Revenue_Item_3, VALUE: N/A
['N/A', 'N/A']
['Revenue_Item_3', 'N/A', 'N/A']

type of key: <class 'str'>, type of value: <class 'str'>
KEY: Sales, VALUE: $391,035M| $383,285M
['$391,035M', ' $383,285M', 'N/A']
['Sales', '$391,035M', ' $383,285M']

type of key: <class 'str'>, type of value: <class 'str'>
KEY: Cost_of_Sales, VALUE: $210,352M| $214,137M
['$210,352M', ' $214,137M', 'N/A']
['Cost_of_Sales', '$210,352M', ' $214,137M']

type of key: <class 

In [10]:
from openai import OpenAI
client = OpenAI()
client.models.list()

SyncPage[Model](data=[Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='text-embedding-ada-002', created=1671217299, object='model', owned_by='openai-internal'), Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system'), Model(id='gpt-4o', created=1715367049, object='model', owned_by='system'), Model(id='gpt-4o-2024-08-06', created=1722814719, object='model', owned_by='system'), Model(id='text-embedding-3-large', created=1705953180, object='model', owned_by='system'), Model(id='gpt-4o-mini-2024-07-18', created=1721172717, object='model', owned_by='system'), Model(id='babbage-002', created=1692634615, object='model', owned_by='system'), Model(id='gpt-4o-2024-11-20', created=1731975040, object='model', owned_by='system'), Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system'), Model(id='o1-mini', created=1725649008, object='model', owned_by='system'), Model(id='davinci-002', created=169263430